# SV-SCN Training on Google Colab

**Complete guide to train 3D shape completion model on free GPU**

⏱️ **Time:** 10 min (quick test) or 2-3 hours (full training)


## ⚙️ Step 1: Check GPU

In [ ]:
# Verify GPU is available
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  GPU not enabled! Go to: Runtime → Change runtime type → GPU")

## 📥 Step 2: Clone Project from GitHub

**IMPORTANT:** Replace `YOUR_GITHUB_USERNAME` with your actual GitHub username!

In [ ]:
# Clone your repository
!git clone https://github.com/YOUR_GITHUB_USERNAME/frozo-3d-model.git

# Navigate to project directory
%cd frozo-3d-model

# Verify structure
print("\n=== Project Contents ===")
!ls -la

print("\n=== Checking svscn module ===")
!ls -la svscn/ || echo "❌ svscn folder not found!"

print("\n=== Checking scripts ===")
!ls -la scripts/ || echo "❌ scripts folder not found!"

## 📦 Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q open3d>=0.17.0 trimesh>=4.0.0 scipy>=1.10.0
!pip install -q objaverse>=0.1.7 tensorboard>=2.14.0

print("✅ Dependencies installed!")

## 🧪 Step 4: Test Model Architecture

In [ ]:
# Quick test to verify imports work
import sys
import torch
from svscn.models import SVSCN
from svscn.config import default_config

print("✅ All imports successful!")
print(f"Model version: {default_config.VERSION}")

# Test model creation
model = SVSCN(num_classes=3)
if torch.cuda.is_available():
    model = model.cuda()
    
print(f"✅ Model created on: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Count parameters
params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {params:,}")

## 📊 Step 5: Generate Training Data

Creates 500 synthetic furniture meshes for training

In [ ]:
# Generate placeholder data
!python -m svscn.data.shapenet \
    --placeholder \
    --output_dir data/shapenet \
    --num_samples 500

print("\n✅ Generated 500 furniture meshes")

In [ ]:
# Convert meshes to point clouds
!python -m svscn.data.preprocess \
    --input_dir data/shapenet \
    --output_dir data/processed \
    --num_points 8192

print("\n✅ Converted to point clouds")

In [ ]:
# Generate training pairs (partial + complete)
!python -m svscn.data.augment \
    --input_dir data/processed \
    --output_dir data/training \
    --views_per_sample 3

print("\n✅ Created training pairs")

# Verify data
!echo "\nTraining data structure:"
!find data/training -type f | head -10

## 🚀 Step 6A: Quick Training (10 epochs, ~10 min)

**Start here to test the pipeline!**

In [ ]:
# Quick test training
!python scripts/train.py \
    --data_dir data/training \
    --epochs 10 \
    --batch_size 32 \
    --checkpoint_dir checkpoints \
    --log_dir logs \
    --device cuda

## 🎯 Step 6B: Full Training (150 epochs, ~2-3 hours)

**Run this for production-quality model**

⚠️ Make sure to keep Colab tab open or it may disconnect!

In [ ]:
# Full training - UNCOMMENT TO RUN
# !python scripts/train.py \
#     --data_dir data/training \
#     --epochs 150 \
#     --batch_size 32 \
#     --checkpoint_dir checkpoints \
#     --log_dir logs \
#     --device cuda

## 📈 Step 7: Monitor Training (TensorBoard)

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs

## 🧪 Step 8: Test Inference

In [ ]:
# Create test input
import numpy as np

partial = np.random.randn(2048, 3).astype(np.float32)
partial = (partial - partial.mean(axis=0)) / partial.std()

np.save('test_partial.npy', partial)
print(f"Created test input: {partial.shape}")

In [ ]:
# Run inference
!python scripts/infer.py \
    --checkpoint checkpoints/best.pt \
    --input test_partial.npy \
    --output completed.npy \
    --class_id 0 \
    --device cuda

print("\n✅ Inference complete!")

## 📊 Step 9: Visualize Results

In [ ]:
# Visualize point clouds
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

partial = np.load('test_partial.npy')
completed = np.load('completed.npy')

fig = plt.figure(figsize=(15, 5))

# Input
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(partial[:, 0], partial[:, 1], partial[:, 2], c='blue', s=1)
ax1.set_title('Input (Partial)', fontsize=14)
ax1.set_box_aspect([1,1,1])

# Output
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(completed[:, 0], completed[:, 1], completed[:, 2], c='green', s=1)
ax2.set_title('Output (Completed)', fontsize=14)
ax2.set_box_aspect([1,1,1])

# Overlay
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(partial[:, 0], partial[:, 1], partial[:, 2], c='blue', s=1, alpha=0.5, label='Input')
ax3.scatter(completed[:, 0], completed[:, 1], completed[:, 2], c='green', s=1, alpha=0.3, label='Output')
ax3.set_title('Comparison', fontsize=14)
ax3.legend()
ax3.set_box_aspect([1,1,1])

plt.tight_layout()
plt.show()

print(f"\nInput: {len(partial)} points")
print(f"Output: {len(completed)} points")

## 💾 Step 10: Download Trained Model

In [ ]:
# Download checkpoint
from google.colab import files

files.download('checkpoints/best.pt')
print("\n✅ Model downloaded!")
print("Use this checkpoint for local inference")

## 🎨 Step 11: Export as 3D Mesh (GLB)

In [ ]:
# Export to GLB format (AR-ready)
!python scripts/infer.py \
    --checkpoint checkpoints/best.pt \
    --input test_partial.npy \
    --output completed.glb \
    --export_mesh \
    --class_id 0 \
    --device cuda

# Download GLB
files.download('completed.glb')
print("\n✅ GLB file downloaded!")
print("View in: https://3dviewer.net or any AR app")

## 📋 Summary

### ✅ What You've Done:
1. Trained a 3D shape completion model on GPU
2. Generated 500 training samples
3. Tested inference pipeline
4. Downloaded trained checkpoint
5. Exported 3D mesh (GLB format)

### 🚀 Next Steps:
- Use `best.pt` locally for inference
- Train longer (150 epochs) for better quality
- Test on real furniture images
- Deploy for production use

### 💻 Local Inference:
```bash
python scripts/infer.py \
    --checkpoint best.pt \
    --input your_data.npy \
    --output result.glb \
    --export_mesh \
    --device cpu
```